# Exploring and clustering neighborhoods in Toronto - Part 2
Coursera capstone course for applied data science

Here we create a dataframe adding the coordinates for each postal code within Toronto

In [6]:
# we install the geocoder package
!pip install geocoder

    100% |████████████████████████████████| 102kB 10.5MB/s a 0:00:01
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda

In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.display import display_html
import geocoder

Here we load the table using requests and BeautifulSoup.  
The first table with the class "wikitable" is the one we are looking for.  
An interesting Beautiful Soup tutorial can be found here https://www.youtube.com/watch?v=ng2o98k983k

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
table = soup.find('table', class_='wikitable')

Now, we load the dataframe using the read_html method from the IPython library.    
Next lines inspired from this tutorial 
https://beenje.github.io/blog/posts/parsing-html-tables-in-python-with-pandas/

In [9]:
# render the table from the HTML string
# display_html(str(table), raw=True)
# load the table in a dataframe
df = pd.read_html(str(table),header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Table cleanup

In [10]:
# rename the columns
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

# delete the rows where the Borough is not assigned
df = df[df.Borough != 'Not assigned']

# group by Borough and combine the Neighborhood
# see here https://stackoverflow.com/questions/17841149/pandas-groupby-how-to-get-a-union-of-strings
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
# https://stackoverflow.com/questions/35068722/pandas-assign-value-of-one-column-based-on-another
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally we print the dimensions of the table

In [11]:
df.shape

(103, 3)

Adding the coordinates to the dataframe using the CSV file.  
The procedure is to load the file and to join the two dataframes using the column Postal Code as key

In [ ]:
# load csv file with the coords
#df_coords = from_csv(...)

# merge the two dataframes
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

df = pd.merge(df, df_coords, on=['PostalCode', 'Postal Code'])